You are using Copilot for free<br>
Responses are limited to **2,000** `code completions` and **50** `chat messages` *per month*.

In [ ]:
# vector embeddings for text using OpenAI's API and langchain framework
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
#